# [Firedraka 安装](https://www.firedrakeproject.org/download.html)

本节安装方式需要计算机可以正常访问网络, 否则请参考第2节 [无网络安装](#无网络安装)

<!--
jupyter nbconvert --to pdf 01_firedrake_install.ipynb

modified title:
https://github.com/jupyter/nbconvert/issues/249#issuecomment-636318699
-->

## Ubuntu

最简单的方式为下载安装脚本 `firedrake-install` 然后使用 python 运行即可. 该方式默认为实数版本.

```bash
curl -O https://raw.githubusercontent.com/firedrakeproject/firedrake/master/scripts/firedrake-install
python3 firedrake-install
```

需要了解更多安装选项, 请查看帮助:
```bash
python3 firedrake-install -h
```

__注__: 有时安装会出现 `pip` 源不能访问的问题, 类似错误信息如下:

```shell
Starting new HTTPS connection (6): pypi.org:443
Could not fetch URL https://pypi.org/simple/pulp/: connection error: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pulp/ (Caused by NewConnectionError('<pip._vendor.urllib3.connection.HTTPSConnection object at 0x7f43dce52bc0>: Failed to establish a new connection: [Errno 101] Network is unreachable')) - skipping
Skipping link: not a file: https://pypi.org/simple/pulp/
Given no hashes to check 0 links for project 'pulp': discarding no candidates
ERROR: Could not find a version that satisfies the requirement PuLP (from versions: none)
ERROR: No matching distribution found for PuLP
```

可以通过设置 `pip` 的源解决, 如更改为 _中科大_ 的源:

```shell
mkdir -p $HOME/.pip && \
cat > $HOME/.pip/pip.conf <<EOF
[global]
index-url = https://pypi.mirrors.ustc.edu.cn/simple
[install]
trusted-host=pypi.mirrors.ustc.edu.cn
EOF
```

### 安装示例 (`real-int32` 和 `real-int32-debug`)

1. 下载安装脚本

```bash
curl -O https://raw.githubusercontent.com/firedrakeproject/firedrake/master/scripts/firedrake-install
```

2. 开启 PETSc debug 选项 (可选)

```bash
DEBUG='-debug'
sed -i.bak -e 's/\(--with-debugging=\)0/\11/g' firedrake-install
```

3. 更新软件包信息

```bash
sudo apt-get update
sudo apt-get install pkg-config # for p4est
```

4. 安装

```bash
PETSC_CONFIGURE_OPTIONS=" \
    --download-fftw --download-mmg \
    --download-p4est --download-parmmg --download-triangle \
    --download-tetgen --download-hpddm --download-libpng \
    --download-slepc  --download-pragmatic" \
python3 firedrake-install --disable-ssh \
    --documentation-dependencies \
    --venv-name $HOME/firedrake/real-int32$DEBUG
```

###  安装示例 (`complex-int64` 和 `complex-int64-debug`)

1. 下载安装脚本

```bash
curl -O https://raw.githubusercontent.com/firedrakeproject/firedrake/master/scripts/firedrake-install
```

2. 开启 PETSc debug 选项 (可选)

```bash
DEBUG='-debug'
sed -i.bak -e 's/\(--with-debugging=\)0/\11/g' firedrake-install
```

3. 更新软件包信息

```bash
sudo apt-get update
sudo apt-get install pkg-config
```

4. 安装

```bash
PETSC_CONFIGURE_OPTIONS=" \
    --download-fftw --download-mmg \
    --download-p4est --download-parmmg --download-triangle \
    --download-tetgen --download-hpddm --download-libpng \
    --download-slepc --download-scalapack --download-mumps" \
python3 firedrake-install --disable-ssh \
    --documentation-dependencies  \
    --petsc-int-type int64 --complex \
    --venv-name $HOME/firedrake/complex-int64$DEBUG
```

### 安装示例 (使用 MKL)

1.  安装 mkl

    a. 添加源
    
    
    ```bash
    wget -O- https://apt.repos.intel.com/intel-gpg-keys/GPG-PUB-KEY-INTEL-SW-PRODUCTS.PUB \
    | gpg --dearmor | sudo tee /usr/share/keyrings/oneapi-archive-keyring.gpg > /dev/null

    echo "deb [signed-by=/usr/share/keyrings/oneapi-archive-keyring.gpg] https://apt.repos.intel.com/oneapi all main" \
    | sudo tee /etc/apt/sources.list.d/oneAPI.list

    sudo apt update
    ```

    b. 安装 MKL 库和头文件
    
    
    ```bash
    # sudo apt install intel-basekit
    sudo apt install intel-oneapi-mkl
    sudo apt install intel-oneapi-mkl-devel
    ```

2. 下载安装脚本

    ```bash
    curl -O https://raw.githubusercontent.com/firedrakeproject/firedrake/master/scripts/firedrake-install
    ```

3. 修改安装脚本


```bash
sed -i.bak -e 's/\(--with-debugging=\)0/\11/g' -e 's/\({0}\/lib\)/\1\/intel64/g' \
    -e 's/\(.*\)\(--C\)\(FLAGS=-I{}\/include\)\(.*\)/\1\2\3\4\n\1\2XX\3\4/' firedrake-install
```
    a. `'s/\(--with-debugging=\)0/\11/g'` for petsc debug
    b. `'s/\({0}\/lib\)/\1\/intel64/g'` for mkl lib
    c. `'s/\(.*\)\(--C\)\(FLAGS=-I{}\/include\)\(.*\)/\1\2\3\4\n\1\2XX\3\4/'` for hpddm with mkl

4. 安装 real-int32
```bash
time PETSC_CONFIGURE_OPTIONS="--download-fftw --download-mmg \
        --download-p4est --download-parmmg --download-triangle \
        --download-tetgen --download-hpddm --download-libpng \
        --download-slepc  --download-pragmatic \
        --with-mkl_pardiso-dir=/opt/intel/oneapi/mkl/latest \
        --with-mkl_cpardiso-dir=/opt/intel/oneapi/mkl/latest" \
python3 firedrake-install --disable-ssh --documentation-dependencies \
    --with-blas=/opt/intel/oneapi/mkl/latest \
    --venv-name firedrake/real-int32-mkl-debug
```

5. 修复 mkl_cpardiso

    错误信息
    
    ```bash
    $ cd petsc/src/binding/petsc4py/demo/kspsolve
    $ python test_mat_ksp.py -pc_type lu -pc_factor_mat_solver_type mkl_cpardiso -ksp_view
    Intel MKL FATAL ERROR: Cannot load symbol MKLMPI_Get_wrappers.
    ```

    a. 打补丁

    ```text
    diff --git a/src/binding/petsc4py/conf/baseconf.py b/src/binding/petsc4py/conf/baseconf.py
    index fdb9cd1a3e..87585c27b6 100644
    --- a/src/binding/petsc4py/conf/baseconf.py
    +++ b/src/binding/petsc4py/conf/baseconf.py
    @@ -207,6 +207,11 @@ class PetscConfig:
             self._configure_ext(extension, petsc_inc, preppend=True)
             self._configure_ext(extension, petsc_lib)

    +        blas_lib = flaglist(self['BLASLAPACK_LIB'])
    +        blas_inc = flaglist(self['BLASLAPACK_INCLUDE'])
    +        self._configure_ext(extension, blas_inc, preppend=True)
    +        self._configure_ext(extension, blas_lib)
    +
         def configure_compiler(self, compiler):
             if compiler.compiler_type != 'unix': return
             getenv = os.environ.get
    ```

    b. 重新编译安装 petsc4py

    ```bash
    cd /home/yzz/firedrake/real-int32-mkl-debug/src/petsc/src/binding/petsc4py/
    PETSC_DIR=$HOME/firedrake/real-int32-mkl-debug/src/petsc PETSC_ARCH=default make clean
    PETSC_DIR=$HOME/firedrake/real-int32-mkl-debug/src/petsc PETSC_ARCH=default \
    /home/yzz/firedrake/real-int32-mkl-debug/bin/python -m pip install \
        --no-build-isolation --no-binary mpi4py,randomgen,islpy,numpy \
        --no-deps -vvv --ignore-installed \
        /home/yzz/firedrake/real-int32-mkl-debug/src/petsc/src/binding/petsc4py/
    ```


6. 安装 complex-int32

```bash
PETSC_CONFIGURE_OPTIONS=" \
    --download-fftw --download-mmg \
    --download-p4est --download-parmmg --download-triangle \
    --download-tetgen --download-hpddm --download-libpng \
    --download-slepc --download-scalapack --download-mumps \
    --with-mkl_pardiso-dir=/opt/intel/oneapi/mkl/latest \
    --with-mkl_cpardiso-dir=/opt/intel/oneapi/mkl/latest" \
python3 firedrake-install --disable-ssh \
    --documentation-dependencies  \
    --with-blas=/opt/intel/oneapi/mkl/latest --complex \
    --venv-name firedrake/complex-int32-mkl-debug
```

可同上修复 `mkl_cpardiso`

### 测试

```shell
source firedrake/bin/activate
cd $VIRTUAL_ENV/src/firedrake
pytest tests/regression/ -k "poisson_strong or stokes_mini or dg_advection"
```

## Windows

安装 `WSL` (适用于 Linux 的 Windows 子系统, Windows Subsystem for Linux).
默认情况下，安装的系统为 `Ubuntu`. 

### WSL 安装

https://docs.microsoft.com/zh-cn/windows/wsl/install

### Firedrake 安装
按照 Ubuntu 安装方式

## MacOS

先安装 `Homebrew` (https://brew.sh/), 然后使用 `Homebrew` 安装 `python3`, 之后类似于 Ubuntu 直接安装 `firedrake`

## Linux Server

若服务器不能访问网络, 请参考下节: [无网络安装](#无网络安装).

### Spack I
`Firedrake` 团队提供了基于 `Spack` (HPC 上的包管理器) 安装的方式.

详见 : https://github.com/firedrakeproject/firedrake-spack

### Spack II

使用 `spack` 安装依赖包, 然后类似于 `Ubuntu` 方式安装 (需要禁用包管理器： `--no-package-manager`)

可参考如下脚本:

https://raw.githubusercontent.com/lrtfm/notes-for-firedrake/main/scripts/spack-firedrake.py

### Docker

1. `Firedrake` 团队构建的镜像:

    https://hub.docker.com/u/firedrakeproject.

2. 我构建的镜像:

    https://hub.docker.com/r/lrtfm/firedrake
    
#### TODO: Docker 镜像瘦身

Docker 镜像过于庞大, 可以考虑删除一些不必要的文件:
```bash
firedrake=$HOME/firedrake
rm -rf $HOME/.cache/pip
find $firedrake -name ".git" | xargs rm -rf
find $firedrake -name "*.o" | xargs rm

rm -rf $firedrake/src/{libspatialindex,libsupermesh}

rm -rf $firedrake/src/{petsc,slepc}/src

find $firedrake -name "doc" | xargs rm -rf
find $firedrake -name "docs" | xargs rm -rf
```

```bash
docker export
docker import
```

# 无网络安装

某些 HPC 上不能访问网络, 要在无网络访问的计算节点上安装 `firedrake`, 需要使用 `spack` 的镜像功能. 


假设本地主机可以访问网络 ( `github` 等). 下面我们以安装软件在 `$HOME/opt` 目录为例.


若登录节点可以访问网络, 以下在本地主机的操作可以在登陆节点执行. 
一般 HPC 是共享硬盘存储的, 此时就不需要对下载内容进行打包上传了.

__注__: `spack` 和 `firedrake` 均在持续开发中, 安装出现错误很常见, 此时可以在 `slack` 或 `github` 上咨询或提问:



__注__: 下面多行命令块中各行之间使用了 "`&& \`" 连接, 直接拷贝多行到终端输入回车即可.


Reference:

1. spack install:

    + https://spack.readthedocs.io/en/latest/getting_started.html#installation

2. spack mirror: 
    
    + https://spack.readthedocs.io/en/latest/bootstrapping.html#creating-a-mirror-for-air-gapped-systems
    + https://spack.readthedocs.io/en/latest/mirrors.html#mirror-environment
    + https://spack.readthedocs.io/en/latest/mirrors.html#mirror-files

3. firedrake spack:

    + https://github.com/firedrakeproject/firedrake-spack
    + https://hackmd.io/@TzVnFeL0TMCb3FaAi9qYBA/ByaRskMQ5

## 本地 ( 可以访问网络的主机 )

### 创建安装目录

```bash
mkdir -p $HOME/opt
```

### 下载安装 spack

```bash
cd $HOME/opt && \
git clone -c feature.manyFiles=true https://github.com/spack/spack.git && \
source $HOME/opt/spack/share/spack/setup-env.sh
```
     
__注1__: 当前使用镜像需要打补丁 (for bootstrap mirrors) https://github.com/spack/spack/pull/34552

```bash
pushd $HOME/opt
wget https://patch-diff.githubusercontent.com/raw/spack/spack/pull/34552.patch
popd
pushd $HOME/opt/spack
spack checkout -b local-patch-mirrors
spack am 34552.patch
popd
```
  
__注2__: 添加下面命令到文件 `$HOME/.bashrc` 中, 用于添加 spack 的 shell 支持, 使得每次开启终端都可以使用 spack.
```bash
source $HOME/opt/spack/share/spack/setup-env.sh
```

### 创建 bootstrap 镜像

```bash
spack bootstrap mirror --binary-packages $HOME/opt/bootstrap
```

The output looks like:
```text
==> Adding "clingo-bootstrap@spack+python %gcc target=x86_64" and dependencies to the mirror at /home/xyz/opt/bootstrap/bootstrap_cache

==> Adding "gnupg@2.3: %gcc target=x86_64" and dependencies to the mirror at /home/xyz/opt/bootstrap/bootstrap_cache
==> Adding "patchelf@0.13.1: %gcc target=x86_64" and dependencies to the mirror at /home/xyz/opt/bootstrap/bootstrap_cache
==> Adding "gnuconfig" and dependencies to the mirror at /home/xyz/opt/bootstrap/bootstrap_cache
==> Adding binary packages from "https://github.com/spack/spack-bootstrap-mirrors/releases/download/v0.4/bootstrap-buildcache.tar.gz" to the mirro
r at /home/xyz/opt/bootstrap/bootstrap_cache

To register the mirror on the platform where it's supposed to be used, move "/home/xyz/opt/bootstrap" to its final location and run the following
command(s):

  % spack bootstrap add --trust local-sources <final-path>/metadata/sources
  % spack bootstrap add --trust local-binaries <final-path>/metadata/binaries
```

### 打包 spack 文件和镜像, 用于服务器安装
```bash
tar -czvf spack.tar.gz spack
tar -czvf bootstrap.tar.gz bootstrap
```
        
### 下载 firedrake-spack 仓库

<!--
等待 `firedrake-spack` 修复
```bash
cd $HOME/opt && \
git clone https://github.com/firedrakeproject/firedrake-spack
```
-->

```bash
cd $HOME/opt && \
git clone https://github.com/lrtfm/firedrake-spack.git && \
pushd firedrake-space && \
git checkout lrtfm/air-gapped-install && \
popd
```
                
### 打包 firedrake-spack 文件, 用于服务器安装
```bash
tar -czvf firedrake-spack.tar.gz firedrake-spack
```
        
### 添加该仓库到 `spack`
```bash
spack repo add firedrake-spack
```

### 检查 spack 安装情况
现在运行 `spack info py-firedrake` 查看 firedrake-spack 仓库是否添加成功
```shell
$ spack info py-firedrake
PythonPackage:   py-firedrake

Description:
    Firedrake is an automated system for the portable solution of partial
    differential equations using the finite element method (FEM)

Homepage: https://firedrakeproject.org

Preferred version:
    develop    [git] https://github.com/firedrakeproject/firedrake.git on branch master

Safe versions:
    develop    [git] https://github.com/firedrakeproject/firedrake.git on branch master

Deprecated versions:
    None

Variants:
    Name [Default]               When    Allowed values    Description
    =========================    ====    ==============    ===============================================

    64-bit-indices [off]         --      on, off           Install PETSc using 64bit indices
    build_system [python_pip]    --      python_pip        Build systems supported by the package
    complex [off]                --      on, off           Install Firedrake in complex mode
    minimal-petsc [off]          --      on, off           Build PETSc with minimal packages for Firedrake
    slepc [off]                  --      on, off           Install SLEPc and slepc4py

Build Dependencies:
    eigen            mpi            py-cython  py-h5py        py-mpi4py    py-pip        py-pyadjoint  py-scipy       py-sympy  py-vtk    slepc
    libspatialindex  petsc          py-fiat    py-islpy       py-numpy     py-pkgconfig  py-pyop2      py-setuptools  py-tsfc   py-wheel
    libsupermesh     py-cachetools  py-finat   py-matplotlib  py-petsc4py  py-progress   py-requests   py-slepc4py    py-ufl    python

Link Dependencies:
    eigen  libspatialindex  libsupermesh  mpi  petsc  python  slepc

Run Dependencies:
    eigen            petsc          py-finat       py-mpi4py    py-pip        py-pyop2         py-scipy       py-tsfc  slepc
    libspatialindex  py-cachetools  py-h5py        py-nbval     py-pkgconfig  py-pytest        py-setuptools  py-ufl
    libsupermesh     py-cython      py-islpy       py-numpy     py-progress   py-pytest-xdist  py-slepc4py    py-vtk
    mpi              py-fiat        py-matplotlib  py-petsc4py  py-pyadjoint  py-requests      py-sympy       python

```

当前 `$HOME/opt` 目录文件如下:
```shell
$ ls -lha
total 214M
drwxrwxr-x  4 z2yang z2yang  112 Oct 30 15:17 .
drwxrwxr-x  3 z2yang z2yang   47 Oct 30 15:02 ..
drwxrwxr-x  5 z2yang z2yang  204 Oct 30 15:16 firedrake-spack
-rw-rw-r--  1 z2yang z2yang 211K Oct 30 15:17 firedrake-spack.tar.gz
drwxrwxr-x 10 z2yang z2yang 4.0K Oct 30 15:17 spack
-rw-rw-r--  1 z2yang z2yang 214M Oct 30 15:15 spack.tar.gz
```

## 计算节点 ( 不可以访问网络的主机 )


1. 在集群安装时, 下面安装操作需要在计算节点进行. 在使用 `slurm` 的系统上可以使用 `srun` 启动交互终端, 如:

    ```bash
    srun -p xahctest --pty --export=ALL -N 1 -n 64 --exclusive /bin/bash
    ```
    
    或者使用 `salloc` 分配资源后, 使用 `ssh` 登录到计算节点:
    
    ```bash
    salloc -p xahctest -N 1 -n 4
    ```


2. 在使用 `slurm` 的系统上, 需要添加 `slurm` 为外部包:

    ```bash
    spack external find slurm
    ```

<!--
```bash
cd /home/z2yang/z2yang/server2
export HOME=`pwd`
export TERM=xterm
export PS1=' (server2) \w $ '
mkdir opt
cd opt
cp ../../local/opt/firedrake-spack.tar.gz .
cp ../../local/opt/spack.tar.gz .
```
-->

3. 编译器要求:

    1. 由于 `openblas@0.3.12` 在使用某些 `gcc@7.3.1` 会编译失败, 下面我们使用编译器 `gcc@9.4.0`. 
       另一种方法是修改 `openblas` 的冲突规则, 参考 [spack-pr-3443](https://github.com/spack/spack/pull/34443).
       因为 Amazon Linux GCC 7.3.1 包含 [gcc-bug-87467](https://gcc.gnu.org/bugzilla/show_bug.cgi?id=87467) 的补丁,
       `spack` 更改了 `openblas` 和 `gcc@7` 的冲突规则 [spack-pr-3443](https://github.com/spack/spack/pull/34443).
       然而某些主机上 GCC 7.3.1 并不包含该补丁, 从而导致编译失败.
    
    2. 由于 `gcc@9.4.0` 编译 `llvm@14.0.1` 同样会失败, 所以我们使用 `llvm@12.0.1`. 同样可以修改冲突规则.

### 安装 spack

1. 创建安装目录

    ```bash
    mkdir -p $HOME/opt
    ```

2. 上传文件

    使用 ftp 等工具上传本地的 `firedrake-spack.gz`, `spack.tar.gz` 和 `bootstrap.tar.gz` 到服务器的 `$HOME/opt` 目录.

3. 解压安装 spack 和 bootstrap

    ```bash
    cd $HOME/opt && \
    tar -zxf spack.tar.gz && \
    tar -zxf bootstrap.tar.gz && \
    source $HOME/opt/spack/share/spack/setup-env.sh && \
    spack bootstrap add --trust local-sources $HOME/opt/bootstrap/metadata/sources && \
    spack bootstrap add --trust local-binaries $HOME/opt/bootstrap/metadata/binaries
    ```


    __注1__: 添加下面命令到文件 `$HOME/.bashrc` 中, 用于添加 `spack` 的 `shell` 支持, 使得每次开启终端都可以使用 `spack`.

    ```bash
    source $HOME/opt/spack/share/spack/setup-env.sh
    ```

    __注2__: 某些工作站上 `/tmp` 目录内容没有执行权限, 需要更改 `spack` 的构建目录配置如下.

    ```bash
    mkdir -p $HOME/.spack && \
    cat > $HOME/.spack/config.yaml <<EOF
    config:
      build_stage:
        - \$user_cache_path/stage
    EOF
    ```

4. 解压安装 `firedrake-spack` 仓库

    ```bash
    cd $HOME/opt && \
    tar -zxf firedrake-spack.tar.gz && \
    spack repo add firedrake-spack
    ```

### 准备 `firedrake` 安装环境

1. 创建 `spack` 环境

    ```bash
    FIREDRAKE_ENV_NAME=firedrake-complex-int64 && \
    spack env create -d $FIREDRAKE_ENV_NAME && \
    spack env activate -p $FIREDRAKE_ENV_NAME && \
    spack -e $SPACK_ENV config add concretizer:unify:true
    ```

2. 添加软件包 (可根据需要添加或删减, 这里以 `complex+int64` 为例)

    ```bash
    spack add python py-firedrake@develop%gcc +64-bit-indices+complex ^mpich ^openblas \
        ^petsc+mumps+scalapack+int64+complex+libyaml+parmmg+mmg ^llvm@12.0.1 \
        ^hypre+complex+int64+superlu-dist && \
    spack add py-pygmsh py-meshio py-tqdm py-pyyaml
    ```

    __注1__: 需要显式添加 python, 不然 `view` 中 `python` 是链接而不是拷贝, 
    将导致 `firedrake` 找不到 `spatialindex`: https://github.com/spack/spack/issues/32456
    
    __注2__: 在使用 slurm 提交作业的集群(HPC)上, 需要先添加 slurm 为外部包,
    然后更改 `^mpich` 为 `^mppich +slurm` 或者 `^mpich +slurm pmi=xxx` ( xxx 为 pmi 或 pmi2 ).
    Reference: https://slurm.schedmd.com/mpi_guide.html#mpich

3. 运行 `concretize` (spack 生成软件依赖关系)

    ```bash
    spack concretize -f 2>&1 | tee $SPACK_ENV/spack-firedrake-concretize.log
    ```

4. 查看 `$SPACK_ENV` 目录, 有如下内容

    ```bash
    $ ls -la $SPACK_ENV
    total 620
    drwxrwxr-x 3 z2yang z2yang    118 Oct 30 16:01 .
    drwxrwxr-x 5 z2yang z2yang    147 Oct 30 15:33 ..
    drwxrwxr-x 4 z2yang z2yang     89 Oct 30 16:01 .spack-env
    -rw-rw-r-- 1 z2yang z2yang  54343 Oct 30 16:01 spack-firedrake-concretize.log
    -rw-rw-r-- 1 z2yang z2yang 572917 Oct 30 16:01 spack.lock
    -rw-rw-r-- 1 z2yang z2yang    457 Oct 30 16:01 spack.yaml
    ```

### 本地创建镜像 (可以访问网络的主机)

通过 `spack.lock` 文件在本地创建 firedrake 环境, 构建镜像, 并上传服务器.

1. 下载 `spack.lock` 文件到本地文件夹 `$HOME/opt`.

2. 运行如下命令创建镜像 (创建镜像需要 10min 左右)

    ```bash
    cd $HOME/opt && \
    spack env create -d firedrake-mirror-env spack.lock && \
    spack env activate -p ./firedrake-mirror-env && \
    time spack mirror create -a -d spack-firedrake-mirror 2>&1 | tee creat-mirror.logs
    ```
    
    结束后会有如下输出
    
    ```bash
    ==> Summary for mirror in file:///home/z2yang/z2yang/local/opt/spack-firedrake-mirror
    ==> Archive stats:
        0    already present
        244  added
        0    failed to fetch.

    real    10m56.048s
    user    1m1.559s
    sys     0m13.604s
    ```

    如果有文件获取失败, 先删除缓存, 然后重新创建镜像:
    
    ```bash
    spack clean -ds && \
    time spack mirror create -a -d spack-firedrake-mirror 2>&1 | tee creat-mirror.logs
    ```

3. 打包镜像

    ```bash
    tar -czvf spack-firedrake-mirror.tar.gz spack-firedrake-mirror
    ```

### 添加镜像

1. 上传镜像
    
    使用 ftp 工具上传 `spack-firedrake-mirror.tar.gz` 到服务器上 `$HOME/opt` 目录

2. 解压镜像

    ```bash
    tar -xzvf spack-firedrake-mirror.tar.gz
    ```

3. (计算节点)添加镜像路径到 spack

    ```shell
    cat > $HOME/.spack/mirrors.yaml <<EOF
    mirrors:
      local_filesystem: file://$HOME/opt/spack-firedrake-mirror
    EOF
    ```
    
4. (计算节点) 查看镜像是否添加成功, 运行 `spack mirror lsit` 应该有如下信息  

    ```bash
    $ spack mirror list
    local_filesystem    file://<your-home-path>/opt/spack-firedrake-mirror
    spack-public        https://mirror.spack.io
    ```

### 安装 Firedrake

1. 运行 develop 命令以避免一些错误

    ```bash
    spack develop py-firedrake@develop && \
    spack develop libsupermesh@develop && \
    spack develop petsc@develop && \
    spack develop py-fiat@develop && \
    spack develop py-finat@develop && \
    spack develop py-islpy@develop && \
    spack develop py-petsc4py@develop && \
    spack develop py-pyadjoint@develop && \
    spack develop py-pyop2@develop && \
    spack develop py-coffee@develop && \
    spack develop py-loopy@develop && \
    spack develop py-cgen@develop && \
    spack develop py-codepy@develop && \
    spack develop py-genpy@develop && \
    spack develop py-tsfc@develop && \
    spack develop py-ufl@develop
    ```
    
    __注__: 安装32位版本是需要添加下面命令:
    ```bash
    spack develop chaco@petsc
    ```


2. 安装

   在服务器端运行下面命令安装 (第一次安装需要等待一段时间 1-2hour, 中间可能会失败, goog luck!)

    ```shell
    spack concretize -f 2>&1 | tee $SPACK_ENV/spack-firedrake-develop.log && \
    time spack install --fail-fast --show-log-on-error \
        --log-file $SPACK_ENV/spack-firedrake-install.log
    ```
    
    最后几行输出如下:

    ```shell
    [+] /home/z2yang/z2yang/server2/opt/spack/opt/spack/linux-ubuntu22.04-cascadelake/gcc-11.3.0/py-firedrake-develop-il3tmyuhh37rnaww3u2yxhxcqawp3hh6
    ==> Updating view at /home/z2yang/z2yang/server2/opt/firedrake-complex-int64/.spack-env/view

    real    184m44.242s
    user    836m56.348s
    sys     97m42.901s
    ```

3. 取消激活环境

    ```shell
    despacktivate
    ```
    
    会有如下 Warning 忽略即可

    ```shell
    $ despacktivate
    ==> Warning: Skipping reversal of unreversable operation<class 'spack.util.environment.UnsetEnv'> PETSC_ARCH
    ==> Warning: Skipping reversal of unreversable operation<class 'spack.util.environment.UnsetEnv'> PETSC_ARCH
    ==> Warning: Skipping reversal of unreversable operation<class 'spack.util.environment.UnsetEnv'> PETSC_ARCH
    ==> Warning: Skipping reversal of unreversable operation<class 'spack.util.environment.UnsetEnv'> PETSC_ARCH
    ```

### 使用
1. 激活环境

    ```shell
    cd $HOME/opt && \
    spack env activate -p $FIREDRAKE_ENV_NAME
    ```

2. 测试

    ```shell
    cd $SPACK_ENV/py-firedrake && \
    pytest tests/regression/ -k "poisson_strong or stokes_mini or dg_advection"
    ```

# Colab 上尝试 Firedrake 

Colab 是 Colaboratory 的简称,  (可以看作是在线版 Jupyter, 浏览器中编写和执行 Python 代码)

[FEM on Colab](https://fem-on-colab.github.io/index.html) 支持在 Colab 上安装 [FEniCS, FEniCSx, Firedrake, NGSolve, gmsh](https://fem-on-colab.github.io/packages.html)



## 导入 Package

### Firedrake

大概 3 分钟

```python
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" \
        -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake
```

### Gmsh
```python
try:
    import gmsh
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" \
        -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh
```

## 示例

https://colab.research.google.com/drive/1gM3zMWTskH7XyDi1yJL76BPFnOJjSdYh?usp=sharing

# 问题求助

## spack

    1. https://spackpm.slack.com/
    2. https://groups.google.com/g/spack

## firedrake


__官方文档__: 

+ https://www.firedrakeproject.org/documentation.html

__Github 仓库 (issues and discussions)__: 

+ https://github.com/firedrakeproject/firedrake

+ https://github.com/firedrakeproject/firedrake/issues

+ https://github.com/firedrakeproject/firedrake/discussions

__Slack 讨论组__:

+ https://firedrakeproject.slack.com

邮件列表: 

+ https://mailman.ic.ac.uk/mailman/listinfo/firedrake

# 其他有限元工具

+ [FEniCSx](https://fenicsproject.org/)

+ [NgSolve](https://ngsolve.org/)

+ [deal.II](https://github.com/dealii/dealii)

+ [libMesh](https://libmesh.github.io/)

+ [FreeFEM](https://freefem.org/)

+ [Dune](https://www.dune-project.org/modules/dune-fem/)